In [10]:
%pip install requests pandas beautifulsoup4 lxml flask

  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [flask]
Note: you may need to restart the kernel to use updated packages.


In [1]:
# pip install requests pandas beautifulsoup4 lxml
import re
from datetime import date, timedelta
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup

WIKI_PAGE = "Opinion_polling_for_the_next_United_Kingdom_general_election"
HEADERS = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119 Safari/537.36"}
DEFAULT_YEAR = 2025
RAW_OUT  = Path("uk_polling_2025_national.csv")
CLEAN_OUT = Path("uk_polling_2025_national_dates_for_chart.csv")

def fetch_html():
    for url in [
        f"https://en.wikipedia.org/api/rest_v1/page/html/{WIKI_PAGE}",
        f"https://en.wikipedia.org/w/index.php?title={WIKI_PAGE}&printable=yes",
        f"https://en.wikipedia.org/wiki/{WIKI_PAGE}",
    ]:
        r = requests.get(url, headers=HEADERS, timeout=30)
        if r.ok: return r.text
    raise RuntimeError("Could not fetch Wikipedia page.")

def extract_2025_table_html(html):
    soup = BeautifulSoup(html, "lxml")
    h = soup.find(lambda t: t.name in ("h2","h3") and (t.get("id")=="2025" or t.get_text(strip=True)=="2025"))
    if not h: raise RuntimeError("2025 heading not found.")
    nxt = h.find_next()
    while nxt and not (nxt.name=="table" and "wikitable" in (nxt.get("class") or [])):
        nxt = nxt.find_next()
    if not nxt: raise RuntimeError("2025 wikitable not found.")
    return str(nxt)

def table_to_df(table_html):
    try:
        df = pd.read_html(table_html, header=[0,1])[0]
        df.columns = [c[0] for c in df.columns]
    except Exception:
        df = pd.read_html(table_html, header=0)[0]
    # drop duplicate header rows if any
    header_like = tuple(df.columns.tolist())
    mask = df.apply(lambda r: tuple(map(str, r.values)) == header_like, axis=1)
    df = df.loc[~mask].copy()
    df.columns = [str(c).strip() for c in df.columns]
    # order important columns if present
    wanted = ["Dates conducted","Pollster","Client","Area","Sample size",
              "Lab","Con","Ref","LD","Grn","SNP","PC","Others","Lead"]
    cols = [c for c in wanted if c in df.columns] + [c for c in df.columns if c not in wanted]
    return df[cols]

# --- parse end of fieldwork to a single date ---
MONTHS = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,
          "jul":7,"aug":8,"sep":9,"sept":9,"oct":10,"nov":11,"dec":12}

def to_end_of_fieldwork(s: str):
    if pd.isna(s): return pd.NaT
    t = str(s).replace("–","-").replace("—","-").replace("\xa0"," ").strip()
    # cases: "26-27 Oct", "26 Sep - 3 Oct", "8 Oct"
    m = re.search(r"(\d{1,2})\s+([A-Za-z]{3,})$", t)
    if not m: return pd.NaT
    d, mon = m.groups()
    mon = mon.lower()
    mon = "sept" if mon.startswith("sept") else mon[:3]
    try:
        return pd.Timestamp(year=DEFAULT_YEAR, month=MONTHS[mon], day=int(d))
    except Exception:
        return pd.NaT

def clean_pollster(s: str):
    if pd.isna(s): return s
    return re.sub(r"\s*\[[^\]]*\]", "", str(s)).strip()

# ---- run ----
html = fetch_html()
tbl_html = extract_2025_table_html(html)
df = table_to_df(tbl_html)
df.to_csv(RAW_OUT, index=False)

end = df["Dates conducted"].apply(to_end_of_fieldwork)
df["date"] = end.dt.date
df["year"] = end.dt.year
if "Pollster" in df.columns:
    df["Pollster"] = df["Pollster"].apply(clean_pollster)

front = [c for c in ["date","year","Pollster"] if c in df.columns]
rest = [c for c in df.columns if c not in front]
df = df[front + rest]
df.to_csv(CLEAN_OUT, index=False)
print("Wrote:", CLEAN_OUT)


Wrote: uk_polling_2025_national_dates_for_chart.csv


/var/folders/wm/rntml9cj7270lh5ry0v4gjhc0000gn/T/ipykernel_20785/1058893380.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html, header=[0,1])[0]


In [2]:
import pandas as pd
import re

INFILE  = "uk_polling_2025_national_dates_for_chart.csv"   # or your v2 file
OUTFILE = "uk_polling_2025_national_dates_for_chart_v2.csv"

df = pd.read_csv(INFILE, dtype=str)

party_cols = ["Lab","Con","Ref","LD","Grn","SNP","PC","Others"]
num_cols = ([c for c in ["Sample size"] if c in df.columns] +
            [c for c in party_cols if c in df.columns] +
            [c for c in ["Lead"] if c in df.columns])

dash_variants = {"‚Äì","–","—","−","-"}

def clean_numeric_string(s):
    if pd.isna(s): return s
    t = str(s).strip()
    if t in dash_variants: return ""
    t = t.replace("%","").replace(",","")
    if t.lower() == "tie": return ""
    return t

# Clean and coerce
if "Sample size" in df.columns:
    df["Sample size"] = pd.to_numeric(df["Sample size"].apply(clean_numeric_string), errors="coerce")
for c in [x for x in party_cols if x in df.columns]:
    df[c] = pd.to_numeric(df[c].apply(clean_numeric_string), errors="coerce")
if "Lead" in df.columns:
    df["Lead"] = pd.to_numeric(df["Lead"].apply(clean_numeric_string), errors="coerce")

# Drop rows where ALL numeric fields are NaN (text/blank artefacts)
if num_cols:
    df = df[~df[num_cols].isna().all(axis=1)].copy()

# Keep date/year tidy if present
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
if "year" in df.columns:
    df["year"] = pd.to_numeric(df["year"], errors="coerce")

df.to_csv(OUTFILE, index=False)
print("Wrote:", OUTFILE, "| rows:", len(df))


Wrote: uk_polling_2025_national_dates_for_chart_v2.csv | rows: 240


In [5]:
# uk_polls_2024_2025_extract_and_combine.py
# ------------------------------------------------------------
# Fetch 2024 + 2025 national polling tables from Wikipedia,
# clean them, save per-year CSVs, and a combined CSV.
# ------------------------------------------------------------

import re
from pathlib import Path
from datetime import date
import pandas as pd
import requests
from bs4 import BeautifulSoup

WIKI_PAGE = "Opinion_polling_for_the_next_United_Kingdom_general_election"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119 Safari/537.36"
    )
}

OUT_2024 = Path("uk_polling_2024_national_dates_for_chart.csv")
OUT_2025 = Path("uk_polling_2025_national_dates_for_chart.csv")
OUT_COMBINED = Path("uk_polling_2024_2025_national_dates_for_chart.csv")

# ---------------- utilities ----------------

def fetch_wiki_html() -> str:
    for url in [
        f"https://en.wikipedia.org/api/rest_v1/page/html/{WIKI_PAGE}",
        f"https://en.wikipedia.org/w/index.php?title={WIKI_PAGE}&printable=yes",
        f"https://en.wikipedia.org/wiki/{WIKI_PAGE}",
    ]:
        r = requests.get(url, headers=HEADERS, timeout=30)
        if r.ok:
            return r.text
    raise RuntimeError("Could not fetch Wikipedia page (all endpoints failed).")

def extract_year_table_html(full_html: str, year: int) -> str:
    """Find the <h2/3 id='YEAR'> heading, then the first following wikitable."""
    soup = BeautifulSoup(full_html, "lxml")
    h = soup.find(lambda t: t.name in ("h2", "h3") and (t.get("id") == str(year) or t.get_text(strip=True) == str(year)))
    if not h:
        raise RuntimeError(f"Heading for {year} not found.")
    nxt = h.find_next()
    while nxt and not (nxt.name == "table" and "wikitable" in (nxt.get("class") or [])):
        nxt = nxt.find_next()
    if not nxt:
        raise RuntimeError(f"Wikitable for {year} not found.")
    return str(nxt)

def table_html_to_df(table_html: str) -> pd.DataFrame:
    # Try multi-index header (party colours in second row), then fall back
    try:
        df = pd.read_html(table_html, header=[0, 1])[0]
        df.columns = [c[0] for c in df.columns]
    except Exception:
        df = pd.read_html(table_html, header=0)[0]

    # Remove duplicate header rows that sometimes leak into the body
    header_like = tuple(df.columns.tolist())
    mask = df.apply(lambda r: tuple(map(str, r.values)) == header_like, axis=1)
    df = df.loc[~mask].copy()

    df.columns = [str(c).strip() for c in df.columns]
    wanted = ["Dates conducted","Pollster","Client","Area","Sample size",
              "Lab","Con","Ref","LD","Grn","SNP","PC","Others","Lead"]
    ordered = [c for c in wanted if c in df.columns] + [c for c in df.columns if c not in wanted]
    return df[ordered]

# ------------- cleaning helpers -------------

MONTHS = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,
          "jul":7,"aug":8,"sep":9,"sept":9,"oct":10,"nov":11,"dec":12}

DASHES = {"‚Äì","–","—","−","-"}

def end_of_fieldwork(s: str, year: int) -> pd.Timestamp:
    """
    Parse 'Dates conducted' to the last day of fieldwork as a Timestamp.
    Assumes the table is the given year (works for entries like '26–27 Oct',
    '19 Sep – 1 Oct', '8 Oct'). For cross-year spans (e.g., '30 Dec – 3 Jan'),
    the end date will be Jan of `year`, which is what we want for charting.
    """
    if pd.isna(s): return pd.NaT
    t = str(s).replace("\u2013","-").replace("\u2014","-").replace("\xa0"," ").strip()
    # pick the last 'day month' token
    m = re.search(r"(\d{1,2})\s+([A-Za-z]{3,})$", t)
    if not m: return pd.NaT
    d, mon = m.groups()
    mon = mon.lower()
    mon = "sept" if mon.startswith("sept") else mon[:3]
    try:
        return pd.Timestamp(year=year, month=MONTHS[mon], day=int(d))
    except Exception:
        return pd.NaT

def clean_pollster(s: str) -> str:
    if pd.isna(s): return s
    # strip all square-bracketed footnotes like [2], [a], etc.
    return re.sub(r"\s*\[[^\]]*\]", "", str(s)).strip()

def clean_numeric_string(s: str) -> str:
    if pd.isna(s): return s
    t = str(s).strip()
    if t in DASHES: return ""
    t = t.replace("%","").replace(",","")
    if t.lower() == "tie": return ""
    return t

def coerce_numeric_cols(df: pd.DataFrame) -> pd.DataFrame:
    party_cols = ["Lab","Con","Ref","LD","Grn","SNP","PC","Others"]
    if "Sample size" in df.columns:
        df["Sample size"] = pd.to_numeric(df["Sample size"].apply(clean_numeric_string), errors="coerce")
    for c in [x for x in party_cols if x in df.columns]:
        df[c] = pd.to_numeric(df[c].apply(clean_numeric_string), errors="coerce")
    if "Lead" in df.columns:
        df["Lead"] = pd.to_numeric(df["Lead"].apply(clean_numeric_string), errors="coerce")
    # drop rows where ALL numeric fields are NaN (spurious text/blank rows)
    num_cols = ([c for c in ["Sample size"] if c in df.columns] +
                [c for c in party_cols if c in df.columns] +
                [c for c in ["Lead"] if c in df.columns])
    if num_cols:
        df = df[~df[num_cols].isna().all(axis=1)].copy()
    return df

def add_chart_date(df: pd.DataFrame, year: int) -> pd.DataFrame:
    if "Dates conducted" not in df.columns:
        raise ValueError("Expected 'Dates conducted' column.")
    end = df["Dates conducted"].apply(lambda s: end_of_fieldwork(s, year))
    df["date"] = end.dt.date
    df["year"] = end.dt.year
    return df

def clean_year_table(html: str, year: int) -> pd.DataFrame:
    tbl = extract_year_table_html(html, year)
    df = table_html_to_df(tbl)
    # pollster footnotes
    if "Pollster" in df.columns:
        df["Pollster"] = df["Pollster"].apply(clean_pollster)
    # build chart date + year
    df = add_chart_date(df, year)
    # numeric coercions + row pruning
    df = coerce_numeric_cols(df)

    # reorder: put date/year/pollster first
    front = [c for c in ["date","year","Pollster"] if c in df.columns]
    rest = [c for c in df.columns if c not in front]
    return df[front + rest]

# ------------------ main ---------------------

def main():
    html = fetch_wiki_html()

    df_2024 = clean_year_table(html, 2024)
    df_2025 = clean_year_table(html, 2025)

    df_2024.to_csv(OUT_2024, index=False)
    df_2025.to_csv(OUT_2025, index=False)

    combined = pd.concat([df_2024, df_2025], ignore_index=True)
    combined = combined.sort_values("date", kind="mergesort").reset_index(drop=True)
    combined = combined.drop(columns=["year"], errors="ignore")
    combined.to_csv(OUT_COMBINED, index=False)

    print(f"✅ Wrote:\n  - {OUT_2024}\n  - {OUT_2025}\n  - {OUT_COMBINED}  (rows={len(combined)})")

if __name__ == "__main__":
    main()


/var/folders/wm/rntml9cj7270lh5ry0v4gjhc0000gn/T/ipykernel_20785/1746797476.py:55: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html, header=[0, 1])[0]


✅ Wrote:
  - uk_polling_2024_national_dates_for_chart.csv
  - uk_polling_2025_national_dates_for_chart.csv
  - uk_polling_2024_2025_national_dates_for_chart.csv  (rows=293)


/var/folders/wm/rntml9cj7270lh5ry0v4gjhc0000gn/T/ipykernel_20785/1746797476.py:55: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html, header=[0, 1])[0]


In [6]:
# pip install pandas
from pathlib import Path
import pandas as pd

COMBINED = Path("uk_polling_2024_2025_national_dates_for_chart.csv")
YEAR2024  = Path("uk_polling_2024_national_dates_for_chart.csv")

def force_ge_rows_to_election_day(df: pd.DataFrame) -> pd.DataFrame:
    # Make sure date is a datetime (robust to string/object)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    # Detect the general election result rows
    # Wikipedia typically labels these as "2024 general election" (may include suffixes, e.g. "(Survation)")
    has_ge_label = df.get("Pollster", pd.Series([False]*len(df))).astype(str)\
                      .str.contains("2024 general election", case=False, na=False)
    # Keep only national coverage rows (GB or UK) when that column exists
    if "Area" in df.columns:
        is_national = df["Area"].astype(str).str.strip().isin(["GB","UK"])
        mask = has_ge_label & is_national
    else:
        mask = has_ge_label  # fallback if Area column is missing

    # Set the date to the UK general election polling day
    election_day = pd.Timestamp("2024-07-04")
    df.loc[mask, "date"] = election_day
    # Keep year consistent too
    if "year" in df.columns:
        df.loc[mask, "year"] = 2024

    return df

# --- Patch combined ---
dfc = pd.read_csv(COMBINED, dtype=str)
dfc = force_ge_rows_to_election_day(dfc)
# Save in ISO (recommended for plotting)
dfc["date"] = pd.to_datetime(dfc["date"], errors="coerce").dt.date
dfc.to_csv(COMBINED, index=False)
print(f"Updated {COMBINED}")

# --- Patch 2024 file (optional but recommended) ---
if YEAR2024.exists():
    df24 = pd.read_csv(YEAR2024, dtype=str)
    df24 = force_ge_rows_to_election_day(df24)
    df24["date"] = pd.to_datetime(df24["date"], errors="coerce").dt.date
    df24.to_csv(YEAR2024, index=False)
    print(f"Updated {YEAR2024}")


Updated uk_polling_2024_2025_national_dates_for_chart.csv
Updated uk_polling_2024_national_dates_for_chart.csv


In [13]:
# Regenerate chart so hover only shows the hovered series (no unified hover).
# Change hovermode from 'x unified' to 'closest' and keep all other features from v10.
import pandas as pd, numpy as np
from pathlib import Path
import plotly.graph_objects as go

DATAFILE = Path("uk_polling_2024_2025_national_dates_for_chart.csv")
OUT_HTML = Path("uk_polls_lowess.html")

df = pd.read_csv(DATAFILE, dtype=str)
df["date"] = pd.to_datetime(df["date"], errors="coerce")

party_cols_all = ["Lab","Con","Ref","LD","Grn","SNP","PC","Others"]
party_cols = [c for c in party_cols_all if c in df.columns]
for c in party_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=["date"]).sort_values("date").copy()

# Optional libs
have_lowess = False
try:
    from statsmodels.nonparametric.smoothers_lowess import lowess
    have_lowess = True
except Exception:
    have_lowess = False

have_kalman = False
try:
    from statsmodels.tsa.statespace.structural import UnobservedComponents
    have_kalman = True
except Exception:
    have_kalman = False

party_colors = {"Lab":"#E4003B","Con":"#0087DC","Ref":"#12B6CF","LD":"#FF6400",
                "Grn":"#02A95B","SNP":"#FDF38E","PC":"#005B54","Others":"#7f7f7f"}

fig = go.Figure()

pollster = df.get("Pollster", pd.Series([""]*len(df)))
sample   = df.get("Sample size", pd.Series([""]*len(df)))
custom_all = np.stack([pollster.fillna("").astype(str),
                       sample.fillna("").astype(str)], axis=1)

trace_idx = {p: {"points": None, "lowess": None, "rolling": None, "kalman": None} for p in party_cols}

def prep_daily(series):
    s = series.dropna().sort_index()
    if s.empty: return None, None
    s = s.groupby(level=0).mean()
    s = s.asfreq("D").interpolate("time")
    return s.index, s.values

POINTS_OPACITY = 0.22

for party in party_cols:
    # points
    fig.add_trace(go.Scatter(
        x=df["date"], y=df[party], mode="markers",
        name=f"{party} (points)",
        marker=dict(color=party_colors.get(party), opacity=POINTS_OPACITY, size=6),
        customdata=custom_all,
        hovertemplate=(
            "%{x|%d %b %Y}<br>"
            + party + ": %{y:.1f}%<br>"
            "Pollster: %{customdata[0]}<br>"
            "Sample: %{customdata[1]}<extra></extra>"
        )
    ))
    trace_idx[party]["points"] = len(fig.data)-1

    # daily series for smoothing
    d = df[["date", party]].dropna().set_index("date")
    xs, ys = (None, None)
    if len(d) >= 5:
        xs, ys = prep_daily(d[party])

    # LOWESS
    if xs is not None and have_lowess:
        xsec = (pd.to_datetime(xs).astype(np.int64) // 10**9).values
        sm = lowess(ys, xsec, frac=0.25, it=0, return_sorted=False)
        fig.add_trace(go.Scatter(
            x=xs, y=sm, mode="lines", name=f"{party} (LOWESS)",
            line=dict(color=party_colors.get(party), width=2.7),
            hovertemplate="%{x|%d %b %Y}<br>"+party+": %{y:.1f}%<extra></extra>",
            opacity=1.0
        ))
        trace_idx[party]["lowess"] = len(fig.data)-1

    # Rolling
    if xs is not None:
        ys_roll = pd.Series(ys, index=xs).rolling(window=21, min_periods=5, center=True).mean()
        fig.add_trace(go.Scatter(
            x=ys_roll.index, y=ys_roll.values, mode="lines", name=f"{party} (21d rolling)",
            line=dict(color=party_colors.get(party), width=2.7, dash="dot"),
            hovertemplate="%{x|%d %b %Y}<br>"+party+": %{y:.1f}%<extra></extra>",
            opacity=0.0
        ))
        trace_idx[party]["rolling"] = len(fig.data)-1

    # Kalman
    if xs is not None and have_kalman:
        mod = UnobservedComponents(ys, level="local level")
        res = mod.fit(disp=False)
        yk = res.smoothed_state[0]
        fig.add_trace(go.Scatter(
            x=xs, y=yk, mode="lines", name=f"{party} (Kalman)",
            line=dict(color=party_colors.get(party), width=2.7, dash="dash"),
            hovertemplate="%{x|%d %b %Y}<br>"+party+": %{y:.1f}%<extra></extra>",
            opacity=0.0
        ))
        trace_idx[party]["kalman"] = len(fig.data)-1

# Masks
n = len(fig.data)
def vis_mask(selected_parties):
    vis = [False]*n
    for p in selected_parties:
        for key, idx in trace_idx[p].items():
            if idx is not None:
                vis[idx] = True
    return vis

all_parties = party_cols
top5 = [p for p in ["Lab","Con","Ref","LD","Grn"] if p in party_cols]
prog = [p for p in ["Lab","LD","Grn","SNP","PC"] if p in party_cols]
right = [p for p in ["Con","Ref"] if p in party_cols]

mask_all, mask_top5, mask_prog, mask_right = (
    vis_mask(all_parties), vis_mask(top5), vis_mask(prog), vis_mask(right)
)

def opacity_for_method(method):
    op = [None]*n
    for p in party_cols:
        for meth in ["lowess","rolling","kalman"]:
            idx = trace_idx[p][meth]
            if idx is not None:
                op[idx] = 1.0 if meth == method else 0.0
    return op

op_lowess  = opacity_for_method("lowess")
op_rolling = opacity_for_method("rolling")
op_kalman  = opacity_for_method("kalman")

# Dots toggle arrays
def points_opacity_array(value):
    arr = [None]*n
    for p in party_cols:
        idx = trace_idx[p]["points"]
        if idx is not None:
            arr[idx] = value
    return arr

POINTS_OPACITY = 0.22
dots_on  = points_opacity_array(POINTS_OPACITY)
dots_off = points_opacity_array(0.0)

# Init
for i, v in enumerate(mask_all):
    fig.data[i].visible = v

# Layout: hovermode 'closest' so only hovered series shows
start = pd.Timestamp("2024-07-04")
end   = df["date"].max() + pd.Timedelta(days=7)

fig.update_layout(
    title="UK/GB National Polling — Vote Share",
    xaxis_title="Date (end of fieldwork)",
    yaxis_title="Vote share (%)",
    hovermode="closest",  # <— key change
    hoverlabel=dict(namelength=-1),
    template="plotly_white",
    legend_title_text="Series",
    legend=dict(orientation="h", x=0, y=-0.36, xanchor="left", yanchor="top"),
    margin=dict(l=60, r=30, t=60, b=190),
    updatemenus=[
        # Row 1: Method (line opacity only)
        dict(type="buttons", direction="right", x=0, y=-0.10, xanchor="left", yanchor="top",
             buttons=[
                 dict(label="LOWESS",       method="restyle", args=[{"opacity": op_lowess}]),
                 dict(label="Rolling (21d)",method="restyle", args=[{"opacity": op_rolling}]),
                 dict(label="Kalman",       method="restyle", args=[{"opacity": op_kalman}]),
             ], showactive=False, pad={"r":6,"t":6}),
        # Row 2: Presets (visibility)
        dict(type="buttons", direction="right", x=0, y=-0.20, xanchor="left", yanchor="top",
             buttons=[
                 dict(label="All",        method="update", args=[{"visible": mask_all}]),
                 dict(label="Top 5",      method="update", args=[{"visible": mask_top5}]),
                 dict(label="Progressive",method="update", args=[{"visible": mask_prog}]),
                 dict(label="Right",      method="update", args=[{"visible": mask_right}]),
             ], showactive=False, pad={"r":6,"t":6}),
        # Single Dots toggle (far right)
        dict(type="buttons", direction="right", x=0.88, y=-0.20, xanchor="left", yanchor="top",
             buttons=[dict(label="Dots On/Off", method="restyle",
                           args=[{"marker.opacity": dots_off}], args2=[{"marker.opacity": dots_on}])],
             showactive=False, pad={"r":6,"t":6}),
    ]
)

fig.update_xaxes(rangeslider=dict(visible=False), range=[start, end])
fig.write_html(str(OUT_HTML), include_plotlyjs="cdn", full_html=True)
print("Saved:", OUT_HTML)


Saved: uk_polls_lowess.html
